In [1]:
'''
TODO
- https://www.analyticsvidhya.com/blog/2021/10/sentiment-analysis-with-textblob-and-vader/
- incorporate ML prediction (box office profit)
- https://dash.plotly.com/dash-html-components/button
'''
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create stopword list:
stopwords = set(stopwords.words('english'))

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
data = pd.read_csv("/Users/ajaypatel21/Downloads/imdb_details_extd_extra.csv", encoding="ISO-8859-1")
data = data.drop(['Unnamed: 0', 'X'], axis=1)

'''Cleaning release period'''
data['release_period'] = data['release_period'].map({'feb_to_apr':'Feb. to Apr.',
                                                     'may_to_jul':'May to Jul.',
                                                     'nov_to_jan':'Nov. to Jan.',
                                                     'aug_to_oct':'Aug. to Oct.'})

data['languages'] = np.where(data['languages'] == 'English, None', 'English', data['languages'])
data['languages'] = np.where(data['languages'] == 'None, English', 'English', data['languages'])
data['languages'] = np.where(data['languages'] == 'None, French', 'French', data['languages'])
data['languages'] = np.where(data['languages'].str.contains(','), 'Multilingual', data['languages'])
data['languages'] = np.where(data['languages'] == '', 'None', data['languages'])
data['languages'] = np.where(data['languages'] != 'English', 'Multilingual_Foreign', data['languages'])

# '''Randomly selecting one of the listed genres and adding new variable to df'''
# np.random.seed(123)
# genre = []
# data['genres'] = np.where(data['genres'] == '', 'Other', data['genres'])
# data['genres'] = data['genres'].fillna('Other')

# for i in range(0, len(data)):
#     num_genres = len(data.loc[i, 'genres'].split(', '))
#     random_genre_idx = np.random.randint(0, num_genres, 1)
#     genre.append(data.loc[i, 'genres'].split(', ')[random_genre_idx[0]])
    
# data['genre'] = genre
# data['genre'] = np.where(data['genre'].isin(['Drama', 'Comedy', 'Action', 'Adventure', 'Crime',
#                                              'Thriller', 'Romance', 'Horror', 'Mystery', 'Sci-Fi', 
#                                              'Fantasy', 'Biography', 'Animation', 'Family']), 
#                          data['genre'], 'Other')

'''Splitting genres variable so each row is a unique movie-genre combination'''
data = data.set_index([i for i in data.columns if i != 'genres']).apply(
    lambda x: x.str.split(', ').explode()).reset_index()

data['genre'] = np.where(data['genres'].isin(['Drama', 'Comedy', 'Action', 'Adventure', 'Crime',
                                                 'Thriller', 'Romance', 'Horror', 'Mystery', 'Sci-Fi', 
                                                 'Fantasy', 'Biography', 'Animation', 'Family']), 
                          data['genres'], 'Other')



'''Cleaning rating'''
data['rating'] = np.where(data['rating'].isin(['PG', 'PG-13', 'R']), data['rating'], 'Other')

'''Removing stop words from plot'''
data['plot no stop words'] = data['plot'].dropna().apply(
    lambda x: [x for x in word_tokenize(x) if not x.lower() in stopwords])

'''Get the polarity of the plot'''
sia = SentimentIntensityAnalyzer()
data['polarity'] = data['plot'].dropna().apply(lambda x: sia.polarity_scores(x))

'''Each value in data['polarity'] is a dictionary --> separating each sentiment into its own variable'''
data['negative'] = data['polarity'].dropna().apply(lambda x: x['neg'])
data['positive'] = data['polarity'].dropna().apply(lambda x: x['pos'])
data['neutral'] = data['polarity'].dropna().apply(lambda x: x['neu'])
data = data.drop('polarity', axis=1)

data

,id,title,fullTitle,type,year,date,runtime,plot,awards,directors,...,co_size,release_month,release_period,budget_adj,genres,genre,plot no stop words,negative,positive,neutral
0,tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari (1920),Movie,1920,19200227.0,67.0,"Francis, a young man, recalls in his memory th...","Awards, 1 win & 1 nomination",Robert Wiene,...,0,2.0,Feb. to Apr.,284898.0,Horror,Horror,"[Francis, ,, young, man, ,, recalls, memory, h...",0.098,0.104,0.798
1,tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari (1920),Movie,1920,19200227.0,67.0,"Francis, a young man, recalls in his memory th...","Awards, 1 win & 1 nomination",Robert Wiene,...,0,2.0,Feb. to Apr.,284898.0,Mystery,Mystery,"[Francis, ,, young, man, ,, recalls, memory, h...",0.098,0.104,0.798
2,tt0010323,The Cabinet of Dr. Caligari,The Cabinet of Dr. Caligari (1920),Movie,1920,19200227.0,67.0,"Francis, a young man, recalls in his memory th...","Awards, 1 win & 1 nomination",Robert Wiene,...,0,2.0,Feb. to Apr.,284898.0,Thriller,Thriller,"[Francis, ,, young, man, ,, recalls, memory, h...",0.098,0.104,0.798
3,tt0012349,The Kid,The Kid (1921),Movie,1921,19210206.0,68.0,The opening title reads: A comedy with a smile...,"Top rated movie #129 | Awards, 2 wins",Charles Chaplin,...,1,2.0,Feb. to Apr.,3414076.0,Comedy,Comedy,"[opening, title, reads, :, comedy, smile, --, ...",0.070,0.133,0.797
4,tt0012349,The Kid,The Kid (1921),Movie,1921,19210206.0,68.0,The opening title reads: A comedy with a smile...,"Top rated movie #129 | Awards, 2 wins",Charles Chaplin,...,1,2.0,Feb. to Apr.,3414076.0,Drama,Drama,"[opening, title, reads, :, comedy, smile, --, ...",0.070,0.133,0.797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13547,tt9866072,Holidate,Holidate (2020),Movie,2020,20201028.0,104.0,Randomly meeting at a store return counter int...,"Awards, 1 win",John Whitesell,...,1,10.0,Aug. to Oct.,NaN,Comedy,Comedy,"[Randomly, meeting, store, return, counter, in...",0.058,0.145,0.797
13548,tt9866072,Holidate,Holidate (2020),Movie,2020,20201028.0,104.0,Randomly meeting at a store return counter int...,"Awards, 1 win",John Whitesell,...,1,10.0,Aug. to Oct.,NaN,Romance,Romance,"[Randomly, meeting, store, return, counter, in...",0.058,0.145,0.797
13549,tt9893250,I Care a Lot,I Care a Lot (2020),Movie,2020,20210219.0,118.0,"Poised with sharklike self-assurance, Marla Gr...","Awards, 1 win & 6 nominations",J Blakeson,...,1,2.0,Feb. to Apr.,NaN,Comedy,Comedy,"[Poised, sharklike, self-assurance, ,, Marla, ...",0.134,0.152,0.715
13550,tt9893250,I Care a Lot,I Care a Lot (2020),Movie,2020,20210219.0,118.0,"Poised with sharklike self-assurance, Marla Gr...","Awards, 1 win & 6 nominations",J Blakeson,...,1,2.0,Feb. to Apr.,NaN,Crime,Crime,"[Poised, sharklike, self-assurance, ,, Marla, ...",0.134,0.152,0.715


In [3]:
[i for i in data.columns]

['id',
 'title',
 'fullTitle',
 'type',
 'year',
 'date',
 'runtime',
 'plot',
 'awards',
 'directors',
 'writers',
 'stars',
 'companies',
 'languages',
 'rating',
 'imDbRatingCount',
 'imDbRating',
 'metacriticRating',
 'budget',
 'grossUSA',
 'grossWorldwide',
 'keywords',
 'oscar_won',
 'oscar_nom',
 'star_power',
 'dir_pop',
 'dir_pop_fac',
 'wr_pop',
 'max_co_frequency',
 'co_size',
 'release_month',
 'release_period',
 'budget_adj',
 'genres',
 'genre',
 'plot no stop words',
 'negative',
 'positive',
 'neutral']

In [4]:
# '''
# data2 is dataframe where each movie name + genre is a unique combination. Now, we can analyze all
# the genres if we want. 
# '''
# data2 = data.copy().drop(['genre', 'plot no stop words'], axis=1)
# data2 = data2.set_index([i for i in data2.columns if i != 'genres']).apply(
#     lambda x: x.str.split(', ').explode()).reset_index()

# data2['genres'] = np.where(data2['genres'].isin(['Drama', 'Comedy', 'Action', 'Adventure', 'Crime',
#                                                  'Thriller', 'Romance', 'Horror', 'Mystery', 'Sci-Fi', 
#                                                  'Fantasy', 'Biography', 'Animation', 'Family']), 
#                            data2['genres'], 'Other')

# data2

In [5]:
'''Creating dataframe with just positive and negative. Rescoring the values so they sum to 1'''
pos_neg = data[['positive', 'negative']].copy()
pos_neg['pos_neg'] = pos_neg.sum(axis=1)
pos_neg['positive'] = pos_neg['positive'] / pos_neg['pos_neg']
pos_neg['negative'] = pos_neg['negative'] / pos_neg['pos_neg']
pos_neg = pos_neg.drop('pos_neg', axis=1)
pos_neg['genre'] = data['genre']
pos_neg['year'] = data['year']

'''
Separating positive from negative into separate dataframes --> combining the two dataframes into 1,
so we can create a pie chart.
'''
pos = pos_neg[['genre', 'positive', 'year']].rename(columns = {'positive':'score'})
pos['sentiment'] = 'Positive'

neg = pos_neg[['genre', 'negative', 'year']].rename(columns = {'negative':'score'})
neg['sentiment'] = 'Negative'

sentiment = pd.concat([pos, neg]).reset_index().drop('index', axis=1)
sentiment

,genre,score,year,sentiment
0,Horror,0.514851,1920,Positive
1,Mystery,0.514851,1920,Positive
2,Thriller,0.514851,1920,Positive
3,Comedy,0.655172,1921,Positive
4,Drama,0.655172,1921,Positive
...,...,...,...,...
27099,Comedy,0.285714,2020,Negative
27100,Romance,0.285714,2020,Negative
27101,Comedy,0.468531,2020,Negative
27102,Crime,0.468531,2020,Negative


In [ ]:
__name__ = 'main'

app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.H1(
            'IMDb Analysis Dashboard',
            style = {'textAlign':'center'},
        ),
        
        html.H4(
            'Sofia Alcazar, Daniel Kwon, Dylan Jorling, Ajay Patel',
            style = {'textAlign':'center'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown1', 
                                    options = ['From'] + sorted(data['year'].unique()), 
                                    value = 'From'),
            style = {'width':'33%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown2', 
                                    options = ['To'] + sorted(data['year'].unique()), 
                                    value = 'To'),
            style = {'width':'34%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown3', 
                                    options = ['All Genres'] + sorted(data['genre'].unique()), 
                                    value = 'All Genres'),
            style = {'width':'33%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Graph(id = 'graph1')
        ),
        
        html.Div(
            children = [dcc.Graph(id = 'graph2')],
            style = {'width':'50%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = [dcc.Graph(id = 'graph3')],
            style = {'width':'50%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = [dcc.Graph(id = 'graph4')],
            style = {'width':'33%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = [dcc.Graph(id = 'graph5')],
            style = {'width':'34%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = [dcc.Graph(id = 'graph6')],
            style = {'width':'33%', 'display':'inline-block'}
        ),
        
        html.H1(
            'How Much Will a New Movie Make?',
            style = {'textAlign':'center'},
        ),
        
        html.H3(
            'Please select a budget, runtime, release period, and company size.',
            style = {'textAlign':'center'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown4', 
                                    options = ['$'+"{:,}".format(i) for i in 
                                               range(10000000, 410000000, 10000000)], 
                                    value = 'budget', 
                                    placeholder = 'Enter Budget'),
            style = {'width':'25%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown5', 
                                    options = [str(i) + ' minutes' for i in range(30, 210, 30)],
                                    value = 'runtime',
                                    placeholder = 'Runtime'),
            style = {'width':'25%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown6', 
                                    options = ['Feb. to Apr.', 'May to Jul.', 'Aug. to Oct.', 'Nov. to Jan.'], 
                                    value = 'release_period',
                                    placeholder = 'Release Period'),
            style = {'width':'25%', 'display':'inline-block'}
        ),
        
        html.Div(
            children = dcc.Dropdown(id = 'dropdown7', 
                                    options = ['Small', 'Medium', 'Large', 'X-Large'], 
                                    value = 'company_size',
                                    placeholder = 'Company Size'),
            style = {'width':'25%', 'display':'inline-block'}
        ),
        
        html.H3(
            'We predict your movie will make:\n',
            style = {'textAlign':'center'}
        ),
        
        html.H3(
            id = 'prediction1_output',
            style = {'textAlign':'center'}
        )
    ]
)

@app.callback(
    Output('graph1', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph1(year_from, year_to, genre):
    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] <= year_to) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to) & 
                             (data['genre'] == genre)]
      
    trace = px.histogram(
        x = filtered_data['imDbRating'],
        title = "Distribution of IMDb Ratings",
        labels = {'x':'IMDb Ratings',
                  'count':'Count'}
    )
    
    return trace


@app.callback(
    Output('graph2', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph2(year_from, year_to, genre):
    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = sentiment
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = sentiment[(sentiment['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] >= year_from) & (sentiment['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] <= year_to) & (sentiment['genre'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] >= year_from) & (sentiment['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = sentiment[(sentiment['year'] >= year_from) & (sentiment['year'] <= year_to) & 
                             (sentiment['genre'] == genre)]
    
    trace = px.pie(
        filtered_data, values = 'score', names = 'sentiment', color = 'sentiment',
        color_discrete_map = {'Positive':'lightgreen',
                              'Negative':'red'},
        title = "Plot Sentiment"
    )
        
    return trace


@app.callback(
    Output('graph3', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph3(year_from, year_to, genre):
    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] <= year_to) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to) & 
                             (data['genre'] == genre)]

    positive = filtered_data[filtered_data['positive'] > filtered_data['negative']].groupby(
        ['year'])['imDbRating'].mean().reset_index()
    
    negative = filtered_data[filtered_data['positive'] < filtered_data['negative']].groupby(
        ['year'])['imDbRating'].mean().reset_index()
    
    positive['Sentiment'] = 'Positive'
    negative['Sentiment'] = 'Negative'
    
    positive_negative = pd.concat([positive, negative]).reset_index()
        
    trace = px.line(
        x = positive_negative['year'],
        y = positive_negative['imDbRating'],
        color = positive_negative['Sentiment'],
        color_discrete_map = {'Positive':'green',
                              'Negative':'red'},
        title = "IMDb Rating Over Time by Sentiment",
        labels = {'x':'Time',
                  'y':'IMDb Rating'}
    )
    
    return trace


@app.callback(
    Output('graph4', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph4(year_from, year_to, genre):
#     if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
#         filtered_data = data2
#     if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
#         filtered_data = data2[(data2['year'] >= year_from)]
#     if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
#         filtered_data = data2[(data2['year'] <= year_to)]
#     if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
#         filtered_data = data2[(data2['genres'] == genre)]
#     if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
#         filtered_data = data2[(data2['year'] >= year_from) & (data2['year'] <= year_to)]
#     if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
#         filtered_data = data2[(data2['year'] <= year_to) & (data2['genres'] == genre)]
#     if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
#         filtered_data = data2[(data2['year'] >= year_from) & (data2['genres'] == genre)]
#     if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
#         filtered_data = data2[(data2['year'] >= year_from) & (data2['year'] <= year_to) & 
#                              (data2['genres'] == genre)]

    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['genres'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] <= year_to) & (data['genres'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['genres'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to) & 
                             (data['genres'] == genre)]
        
    trace = px.bar(
        x = filtered_data['genres'].value_counts().sort_values(ascending=True),
        y = filtered_data['genres'].value_counts().sort_values(ascending=True).index,
        title = "Most Popular Genres",
        labels = {'x':'Count',
                  'y':'Genre'}
    )
    
    return trace


@app.callback(
    Output('graph5', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph5(year_from, year_to, genre):
    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] <= year_to) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to) & 
                             (data['genre'] == genre)]
        
    words = filtered_data['keywords'].astype(str).apply(lambda x: x.split(',')).explode().reset_index()

    # remove punctuation
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~.'''
    words = words[~words['keywords'].isin(word_tokenize(punc))].reset_index().drop(
        'level_0', axis=1)

    trace = px.bar(
        x = words['keywords'].value_counts().head(10).sort_values(),
        y = words['keywords'].value_counts().head(10).sort_values().index,
        title = "Most Popular Keywords",
        labels = {'x':'Count',
                  'y':'Keywords'}
    )
    
    return trace


@app.callback(
    Output('graph6', 'figure'),
    Input('dropdown1', 'value'),
    Input('dropdown2', 'value'),
    Input('dropdown3', 'value'))

def update_graph6(year_from, year_to, genre):
    if (year_from == 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data
    if (year_from != 'From') & (year_to == 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from)]
    if (year_from == 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] <= year_to)]
    if (year_from == 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre == 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to)]
    if (year_from == 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] <= year_to) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to == 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['genre'] == genre)]
    if (year_from != 'From') & (year_to != 'To') & (genre != 'All Genres'):
        filtered_data = data[(data['year'] >= year_from) & (data['year'] <= year_to) & 
                             (data['genre'] == genre)]
        
    trace = px.scatter(
        x = filtered_data['imDbRating'],
        y = filtered_data['metacriticRating']/10,
        title = "Do IMDb and Metacritic Rate the Same?",
        labels = {'x':'IMDb Rating',
                  'y':'Metacritic Rating'}
    )
    
    return trace


@app.callback(
    Output('prediction1_output', 'children'),
    Input('dropdown4', 'value'),
    Input('dropdown5', 'value'),
    Input('dropdown6', 'value'),
    Input('dropdown7', 'value'))

def update_prediction1(budget, runtime, release_period, company_size):
    if ((budget != 'budget') & (runtime != 'runtime') & 
        (release_period != 'release_period') & (company_size != 'company_size')):
        
        budget_int = int(budget.replace('$','').replace(',',''))
        runtime_int = int(runtime.replace(' minutes', ''))
        
        release_period_converter = {'Feb. to Apr.':1, 
                                    'May to Jul.':2, 
                                    'Aug. to Oct.':3, 
                                    'Nov. to Jan.':4}
        
        company_size_converter = {'Small':1,
                                  'Medium':2,
                                  'Large':3,
                                  'X-Large':4}
        
        output = (budget_int * 10) + (runtime_int * 2) + (release_period_converter[release_period] * 100) + (company_size_converter[company_size] * 100)
                    
        formatted_output = ['$'+"{:,}".format(i) for i in [output]][0]
        
        return '{}'.format(formatted_output)


if __name__ == 'main':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_favicon.ico?v=2.8.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:02] "GET /apple-touch-icon.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 18:04:06] "POST /_dash-update-component HTTP